In [11]:
import torch
from torch import nn
import torch.nn.functional as F
print(torch.cuda.is_available())

True


In [14]:
from transformers import PreTrainedModel, PretrainedConfig, AutoConfig
class MultiImageRegressionConfig(PretrainedConfig):
    def __init__(
        self,
        output_size=3,
        init_checkpoint=None,
        **kwargs,
    ):
        self.output_size=output_size
        self.init_checkpoint=init_checkpoint
        super().__init__(**kwargs)


class ImageMultiRegressionModel(PreTrainedModel):
    config_class=MultiImageRegressionConfig
    def __init__(self, config, loss=nn.MSELoss()):
        super().__init__(config)
        
        self.inner_model = AutoModel.from_pretrained(config.init_checkpoint)
        self.classifier = nn.Linear(self.inner_model.config.hidden_size, config.output_size)
        self.loss=loss
    
    def forward(self, pixel_values, labels=None):
        outputs = self.inner_model(pixel_values=pixel_values)
        cls_output = outputs.last_hidden_state[:, 0, :]  # image embedding
        values = self.classifier(cls_output)
        loss = None
        if labels is not None:
            loss = self.loss(values.view(-1), labels.view(-1))
        return (loss, values) if loss is not None else values

In [15]:
model = ImageMultiRegressionModel.from_pretrained("models/swinv2-base")

In [17]:
model.push_to_hub("mirluvams/swinv2-base-patch4-window16-256-popocatepetl", private=True)

model.safetensors:   0%|          | 0.00/348M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mirluvams/swinv2-base-patch4-window16-256-popocatepetl/commit/cc6beaef4229a04b205080fdf465af5ad18b3f9c', commit_message='Upload model', commit_description='', oid='cc6beaef4229a04b205080fdf465af5ad18b3f9c', pr_url=None, pr_revision=None, pr_num=None)